# Generic syntactic sugar

Some small modern features, which have a taste of generic programming.

## At last !!

When nesting template arguments, no more need to insert a space between subsequent "<" or ">".

## Static assertions

The new declaration `static_assert` is executed during compilation, on the contrary of `assert()` which is runtime. You can customize the error message. It is typically used to check some template argument :

In [1]:
#include <type_traits>
template <typename MyInt>
struct Rational
 {
  static_assert(std::is_integral<MyInt>::value, "Integral required.") ;
  static_assert(sizeof(MyInt)>=4, "Underlying type is too small.") ;
  MyInt numerator ;
  MyInt denominator ;
 } ;

In [2]:
Rational<int> r1 ;
Rational<short> r2 ;
Rational<double> r3 ;

input_line_8:5:3: error: static_assert failed "Underlying type is too small."
  static_assert(sizeof(MyInt)>=4, "Underlying type is too small.") ;
  ^             ~~~~~~~~~~~~~~~~
input_line_9:3:17: note: in instantiation of template class '__cling_N52::Rational<short>' requested here
Rational<short> r2 ;
                ^
input_line_8:4:3: error: static_assert failed "Integral required."
  static_assert(std::is_integral<MyInt>::value, "Integral required.") ;
  ^             ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
input_line_9:4:18: note: in instantiation of template class '__cling_N52::Rational<double>' requested here
Rational<double> r3 ;
                 ^


Interpreter Error: 

In this other example, we want to check the size of an array is not zero:

In [3]:
#include <iostream>

In [5]:
const int n = 0 ;
// ...
static_assert(n>0,"Wrong size for the array.") ;
double values[n] ;

input_line_12:4:1: error: static_assert failed "Wrong size for the array."
static_assert(n>0,"Wrong size for the array.") ;
^             ~~~


Interpreter Error: 

## Explicit instanciation

The intensive use of template items produces "code bloat" : an explosion of the number of types and functions, which the developer is not fully aware of, and the duplication of all this stuff in any object file compiled separately, leading to huge final executables. Since compilers have mostly failed to solve this issue (hopefully the C++20 modules will help), C++11 enable the developer to help by explicitly forbid or trigger a template instanciation.

#### Forbid a template instanciation in the current file

``` cpp
extern template class Point<int> ;
```

#### Trigger  a template instanciation in the current file

``` cpp
template class Point<int> ;
```

## Class template argument deduction (C++17)

Before C++17, when asking for the instanciation of a templated class, the developer had to provide explicitly the template parameters, although they were often quite obvious (see example below). This often leaded to the definition of `m_make*` utility functions, whose only purpose was to allow the parameter deduction by the compiler.  

In [ ]:
#include <utility>

In [ ]:
std::pair<int,double> p1(42, 3.14) ;
auto p2 = std::make_pair(42, 3.14) ;

The parameter deduction is now allowed for contructors, so one can simply write:

In [3]:
std::pair p1(42, 3.14) ;

#### Sources
* [Cpp Reference](https://en.cppreference.com/w/cpp/language/class_template_argument_deduction)

## Variable templates (C++14)

C++14 introduces the possibility to write variable templates. A common example is the PI constant, which can be expressed with different precisions, depending on the underlying floating point type.

In [ ]:
template <typename T>
const T pi = T(3.1415926535897932385) ;

In [ ]:
template <typename T>
T circular_area(T r)
 { return pi<T> * r * r ; }

In [ ]:
#include <iostream>

In [ ]:
std::cout.precision(15) ;
std::cout << "double : " << circular_area<double>(1) << std::endl ;
std::cout << "float  : " << circular_area<float>(1) << std::endl ;
std::cout << "int    : " << circular_area<int>(1) << std::endl ;

Since C++17, in the standard library, variable templates are used to implement some `_v` shortcuts for the boolean type traits. For example :

In [ ]:
template <typename T>
bool is_integral_v = std::is_integral<T>::value ;

## Alias templates

It is now possible, thanks to the keyword `using`, to define an alias for some partial specialization of a class template.

In [1]:
template <typename T, typename U>
class Couple { public : T x ; U y ; } ;

In [2]:
template <typename U>
using IntCouple = Couple<int,U> ;

In [3]:
IntCouple<double> pid ; // same as Couple<int,double>

In [5]:
template <typename T>
using MapInt = std::map<int,T> ;

In [6]:
MapInt<double> md ; // same asstd::map<int,double>

The keyword `using` can and should now be used instead of `typedef`. It is considered more readable. Some examples below:

In [4]:
#include <list>
#include <map>
#include <iostream>

In [7]:
using real = float ; // typedef float real ;
    
using mymap = std::map<std::vector<int>,std::list<float>> ;
using myitr = mymap::const_iterator ;
    
using fptr = void(*)(int) ;

Note: with the development of `auto` and type infering, it becames less and less necessary to use type aliases so to lighten the code.

## Exercise

In the example below, we want to declare the variable `value2` with the same type as `value1`, but `const` removed. We use the type modifier `remove_const` from the standard library.

1. Why do we need `typename` just before `std::remove_const<T>::type` ?
2. May you write an alias template `my_remove_const_t`, which would be equivalent to `std::remove_const<T>::type`, and simplify the first line of `increment_and_display()` ?

In [8]:
%%file tmp.generic-sugar.cpp

#include <iostream>

template< typename T >
void increment_and_display( T & value1 )
 {
  typename std::remove_const<T>::type value2 { value1 } ;
  ++value2 ;
  std::cout<<value2<<std::endl ;
 }

int main()
 {
  const int size = 10 ;
  increment_and_display(size) ;
 }

Writing tmp.generic-sugar.cpp


In [9]:
!rm -f tmp.generic-sugar.exe && g++ -std=c++17 tmp.generic-sugar.cpp -o tmp.generic-sugar.exe

In [10]:
!./tmp.generic-sugar.exe

11


© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)